## Load all libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import random

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [137]:
pd.set_option('display.max_rows', 500)

## Install Geopy to calculate distances between points

In [10]:
!pip install geopy

In [31]:
from geopy import distance, geocoders, Nominatim
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
from geopy.extra.rate_limiter import RateLimiter

Load all Datasets saved on csv files

In [139]:
df_mad_houses=pd.read_csv('df_mad_houses.csv')

In [ ]:
df_mad_over100k=pd.read_csv('df_mad_over100k.csv')

In [ ]:
df_transport_stops=pd.read_csv('transport_stops_madrid.csv')

In [82]:
df_mad_postcode_latlon= pd.read_csv('latlons_by_postcode_madrid.csv')

In [133]:
df_mad_houses=df_mad_houses.drop(columns=['Unnamed: 0','codigopostalid','lon','lat'])

In [135]:
df_mad_houses

,prices,post_code,rooms,bathrooms,size,poblacion,codigopostalid,lat,lon,latlon_merged
0,112.062€,28031,1 hab,1 wc,49 m²,Villa Vallecas,28031,40.377436,-3.624158,"(40.377435628148554, -3.624158084914701)"
1,137.000€,28031,1 hab,1 wc,32 m²,Villa Vallecas,28031,40.377436,-3.624158,"(40.377435628148554, -3.624158084914701)"
2,100.000€,28031,3 hab,1 wc,48 m²,Villa Vallecas,28031,40.377436,-3.624158,"(40.377435628148554, -3.624158084914701)"
3,100.000€,28031,3 hab,1 wc,48 m²,Villa Vallecas,28031,40.377436,-3.624158,"(40.377435628148554, -3.624158084914701)"
4,135.000€,28031,1 hab,1 wc,49 m²,Villa Vallecas,28031,40.377436,-3.624158,"(40.377435628148554, -3.624158084914701)"
...,...,...,...,...,...,...,...,...,...,...
26660,160.000€,28193,4 hab,1 wc,184 m²,Cervera de Buitrago,28193,43.420435,-3.529145,"(43.420435, -3.5291447)"
26661,360.000€,28512,4 hab,2 wc,189 m²,Villar del Olmo,28512,40.343467,-3.241504,"(40.34346737484606, -3.2415042886969347)"
26662,260.000€,28595,4 hab,3 wc,200 m²,Estremera,28595,43.420435,-3.529145,"(43.420435, -3.5291447)"
26663,145.000€,28595,3 hab,3 wc,200 m²,Estremera,28595,43.420435,-3.529145,"(43.420435, -3.5291447)"


In [139]:
df_mad_houses['poblacion'].value_counts()

San Blas                807
Alcalá de Henares       756
Las Rozas de Madrid     613
Torrejón de Ardoz       602
Justicia                599
                       ... 
Valdaracete               1
Cervera de Buitrago       1
Soto del Real             1
Rozas de Puerto Real      1
Villar del Olmo           1
Name: poblacion, Length: 196, dtype: int64

Change the postcode file to include city and country to be able to make searches to the Nominatim API and get the latlons of each postcode

In [82]:
df_mad_postcode_latlon['address']=list(zip(df_mad_postcode_latlon['codigopostalid'],df_mad_postcode_latlon['city'],df_mad_postcode_latlon['country']))

In [111]:
df_mad_postcode_latlon=df_mad_postcode_latlon.drop(columns=['Unnamed: 0','full_address','location'])

In [269]:
user_agent = 'palvgoya@hotmail.com'
geolocator = Nominatim(user_agent=user_agent)

geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
df_mad_postcode_latlon['location'] = df_mad_postcode_latlon['address'].apply(geocode)

df_mad_postcode_latlon['location'] = df_mad_postcode_latlon['location'].apply(lambda loc: tuple(loc.point) if loc else None)

KeyError: 'address'

In [120]:
df_mad_postcode_latlon[['lat', 'lon', 'alt']] = pd.DataFrame(df_mad_postcode_latlon['location'].tolist(), index=df_mad_postcode_latlon.index)  

In [122]:
df_mad_postcode_latlon['latlon_merged']=list(zip(df_mad_postcode_latlon['lat'],df_mad_postcode_latlon['lon']))

In [124]:
df_mad_postcode_latlon.to_csv('latlons_by_postcode_madrid.csv')

In [125]:
df_mad_postcode_latlon

,codigopostalid,city,country,address,location,lat,lon,alt,latlon_merged
0,28001,madrid,spain,"(28001, madrid, spain)","(40.4271819, -3.6872776666649734, 0.0)",40.427182,-3.687278,0.0,"(40.4271819, -3.6872776666649734)"
1,28002,madrid,spain,"(28002, madrid, spain)","(40.44436359786106, -3.671972654609913, 0.0)",40.444364,-3.671973,0.0,"(40.44436359786106, -3.671972654609913)"
2,28003,madrid,spain,"(28003, madrid, spain)","(40.44196013762609, -3.7053510586159732, 0.0)",40.441960,-3.705351,0.0,"(40.44196013762609, -3.7053510586159732)"
3,28004,madrid,spain,"(28004, madrid, spain)","(40.427525349999996, -3.6950212296048797, 0.0)",40.427525,-3.695021,0.0,"(40.427525349999996, -3.6950212296048797)"
4,28005,madrid,spain,"(28005, madrid, spain)","(40.409289037338226, -3.710041004754097, 0.0)",40.409289,-3.710041,0.0,"(40.409289037338226, -3.710041004754097)"
5,28006,madrid,spain,"(28006, madrid, spain)","(40.43383533987043, -3.680541208403145, 0.0)",40.433835,-3.680541,0.0,"(40.43383533987043, -3.680541208403145)"
6,28007,madrid,spain,"(28007, madrid, spain)","(40.4046034, -3.6834474261815178, 0.0)",40.404603,-3.683447,0.0,"(40.4046034, -3.6834474261815178)"
7,28008,madrid,spain,"(28008, madrid, spain)","(40.42691291585265, -3.7198520527710333, 0.0)",40.426913,-3.719852,0.0,"(40.42691291585265, -3.7198520527710333)"
8,28009,madrid,spain,"(28009, madrid, spain)","(40.41956994587277, -3.6757184630348814, 0.0)",40.419570,-3.675718,0.0,"(40.41956994587277, -3.6757184630348814)"
9,28010,madrid,spain,"(28010, madrid, spain)","(40.432961272457646, -3.6994969982495944, 0.0)",40.432961,-3.699497,0.0,"(40.432961272457646, -3.6994969982495944)"


In [126]:
df_mad_postcode_latlon=df_mad_postcode_latlon.drop(columns=['city','country','address','location','alt'])

Merge the houses with the latlons file to get latlons of each house by postcode

In [134]:
df_mad_houses=df_mad_houses.merge(df_mad_postcode_latlon,left_on='post_code',right_on='codigopostalid')

Cleaning the transport stops file to use only the columns we need

In [63]:
df_transport_stops=transport_df= df_transport_stops.drop(columns=['stop_id', 'stop_code', 'stop_name', 'stop_desc', 'zone_id','stop_url', 'location_type', 'parent_station', 'stop_timezone', 'wheelchair_boarding'])

In [64]:
df_transport_stops['stop_loc'] = list(zip(df_transport_stops['stop_lat'], df_transport_stops['stop_lon']))

## Calculate distance of each house to closest transport station

Calculating closest transport stop to center point of each neighbourhood

In [206]:
dist_transport=[]

for a in range(0,26665):
    point = df_mad_houses['latlon_merged'][a]
    dist = df_transport_stops['stop_loc'].apply(lambda x: distance.geodesic(x, point).m).min()
    dist_transport.append(dist)

In [207]:
df_mad_houses['transport']=dist_transport

In [208]:
len(dist_transport)

26665

In [195]:
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Talamanca de Jarama', ['lat','lon','latlon_merged']] = '40.7460256','-3.5129914','(40.7460256, -3.5129914)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Belmonte de Tajo', ['lat','lon','latlon_merged']] = '40.1336305','-3.3398654','(40.1336305, -3.3398654)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Carabaña', ['lat','lon','latlon_merged']] = '40.256626','-3.2347206','(40.256626, -3.2347206)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Robledo de Chavela', ['lat','lon','latlon_merged']] = '40.5009201','-4.2380999','(40.5009201, -4.2380999)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'La Cabrera', ['lat','lon','latlon_merged']] = '40.8651738','-3.6125468','(40.8651738, -3.6125468)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Ribatejada', ['lat','lon','latlon_merged']] = '40.6664949','-3.3919178','(40.6664949, -3.3919178)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Villa del Prado', ['lat','lon','latlon_merged']] = '40.2764752','-4.3048032','(40.2764752, -4.3048032)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Torrelaguna', ['lat','lon','latlon_merged']] = '40.8277273','-3.538293','(40.8277273, -3.538293)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Santa María de la Alameda', ['lat','lon','latlon_merged']] = '40.5958248','-4.2579208','(40.5958248, -4.2579208)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Valdemanco', ['lat','lon','latlon_merged']] = '40.863336950000004','-3.6624598591227406','(40.863336950000004, -3.6624598591227406)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'El Vellón', ['lat','lon','latlon_merged']] = '40.7677965','-3.5820099','(40.7677965, -3.5820099)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Aldea del Fresno', ['lat','lon','latlon_merged']] = '40.3215212','-4.19936','(40.3215212, -4.19936)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Cubas de la Sagra', ['lat','lon','latlon_merged']] = '40.1908352','-3.8378366', '(40.1908352, -3.8378366)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Perales de Tajuña', ['lat','lon','latlon_merged']] = '40.2325718','-3.3517265', '(40.2325718, -3.3517265)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Valdilecha', ['lat','lon','latlon_merged']] = '40.2926854','-3.3002096', '(40.2926854, -3.3002096)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Bustarviejo', ['lat','lon','latlon_merged']] = '40.8586659','-3.7101535', '(40.8586659, -3.7101535)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Getafe', ['lat','lon','latlon_merged']] = '40.3081807','-3.7302679', '(40.3081807, -3.7302679)'

In [211]:
df_mad_houses=df_mad_houses[df_mad_houses['transport']<=50000]

In [205]:
df_mad_houses['latlon_merged']=list(zip(df_mad_houses['lat'],df_mad_houses['lon']))

In [357]:
df_mad_houses.to_csv('df_mad_houses.csv')

## Calculate distance of each house to closest school

In [214]:
df_schools=pd.read_csv('Adresses_Schools_Mad.csv')

In [297]:
geolocator = geocoders.ArcGIS()

#geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
df_schools['location'] = df_schools['address'].apply(geolocator.geocode,timeout=10)

#df_hospitals['location'] = df_mad_postcode_latlon['location'].apply(lambda loc: tuple(loc.point) if loc else None)

KeyError: 'location'

In [316]:
df_schools[['a','latlon']]= pd.DataFrame(df_schools['location'].to_list(),index=df_schools.index)

In [319]:
df_schools=df_schools.drop(columns=['a','address','random1','random2','Domicilio','Código'])

In [321]:
df_schools.to_csv('schools_mad_final.csv')

In [346]:
dist_school=[]

for a in range(0,26600):
    point = df_mad_houses['latlon_merged'][a]
    dist = df_schools['latlon'].apply(lambda x: distance.geodesic(x, point).m).min()
    dist_school.append(dist)

KeyError: 26599

In [347]:
dist_school

[201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,


In [348]:
df_mad_houses['school']=dist_school

In [349]:
df_mad_houses

,prices,post_code,rooms,bathrooms,size,poblacion,lat,lon,latlon_merged,transport,school
0,112.062€,28031,1 hab,1 wc,49 m²,Villa Vallecas,40.3774,-3.62416,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674
1,137.000€,28031,1 hab,1 wc,32 m²,Villa Vallecas,40.3774,-3.62416,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674
2,100.000€,28031,3 hab,1 wc,48 m²,Villa Vallecas,40.3774,-3.62416,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674
3,100.000€,28031,3 hab,1 wc,48 m²,Villa Vallecas,40.3774,-3.62416,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674
4,135.000€,28031,1 hab,1 wc,49 m²,Villa Vallecas,40.3774,-3.62416,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674
...,...,...,...,...,...,...,...,...,...,...,...
26594,175.000€,28721,4 hab,2 wc,156 m²,Cabanillas de la Sierra,40.818,-3.60548,"(40.81798043643363, -3.6054796201378334)",23994.341683,1790.021114
26595,175.000€,28721,4 hab,2 wc,156 m²,Cabanillas de la Sierra,40.818,-3.60548,"(40.81798043643363, -3.6054796201378334)",23994.341683,1790.021114
26596,329.000€,28791,3 hab,2 wc,150 m²,Soto del Real,40.7526,-3.77991,"(40.752559745929474, -3.7799147561052875)",11952.570673,307.756355
26597,360.000€,28512,4 hab,2 wc,189 m²,Villar del Olmo,40.3435,-3.2415,"(40.34346737484606, -3.2415042886969347)",18058.216754,899.348506


## Calculate distance of each house to closest health centre

In [298]:
df_hospitals=pd.read_csv('salud_madrid.csv')

In [293]:
df_hospitals['address']= df_hospitals['address'].str.replace('[', ',')
df_hospitals['address']= df_hospitals['address'].str.replace(']', '')

In [294]:
df_hospitals=df_hospitals.drop(columns=['direccion','random1','random2'])

In [299]:
geolocator = geocoders.ArcGIS()

df_hospitals['location'] = df_hospitals['address'].apply(geolocator.geocode,timeout=10)


KeyError: 'location'

In [286]:
from geopy import geocoders
g = geocoders.ArcGIS()

#create an input address string
#you can also build this by reading from an input database and building a string

inputAddress = 'Prof. Martin Lagos, s/n  ,28040, Madrid, Spain' 

#do the geocode
#location = g.geocode(inputAddress, timeout=10)
location= inputAddress.apply(g.geocode,timeout=10)

#some things you can get from the result
#print(location.latitude, location.longitude)

print(location)


AttributeError: 'str' object has no attribute 'apply'

In [303]:
df_hospitals[['place','latlon']] = pd.DataFrame(df_hospitals['location'].tolist(), index=df_hospitals.index)  

In [306]:
df_hospitals.to_csv('salud_madrid_final.csv')

In [354]:
dist_health_centre=[]

for a in range(0,26599):
    point = df_mad_houses['latlon_merged'][a]
    dist = df_hospitals['latlon'].apply(lambda x: distance.geodesic(x, point).m).min()
    dist_health_centre.append(dist)

In [355]:
df_mad_houses['health_centre']=dist_health_centre

In [356]:
df_mad_houses

,prices,post_code,rooms,bathrooms,size,poblacion,lat,lon,latlon_merged,transport,school,health_centre
0,112.062€,28031,1 hab,1 wc,49 m²,Villa Vallecas,40.3774,-3.62416,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674,892.420899
1,137.000€,28031,1 hab,1 wc,32 m²,Villa Vallecas,40.3774,-3.62416,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674,892.420899
2,100.000€,28031,3 hab,1 wc,48 m²,Villa Vallecas,40.3774,-3.62416,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674,892.420899
3,100.000€,28031,3 hab,1 wc,48 m²,Villa Vallecas,40.3774,-3.62416,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674,892.420899
4,135.000€,28031,1 hab,1 wc,49 m²,Villa Vallecas,40.3774,-3.62416,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674,892.420899
...,...,...,...,...,...,...,...,...,...,...,...,...
26594,175.000€,28721,4 hab,2 wc,156 m²,Cabanillas de la Sierra,40.818,-3.60548,"(40.81798043643363, -3.6054796201378334)",23994.341683,1790.021114,5226.267593
26595,175.000€,28721,4 hab,2 wc,156 m²,Cabanillas de la Sierra,40.818,-3.60548,"(40.81798043643363, -3.6054796201378334)",23994.341683,1790.021114,5226.267593
26596,329.000€,28791,3 hab,2 wc,150 m²,Soto del Real,40.7526,-3.77991,"(40.752559745929474, -3.7799147561052875)",11952.570673,307.756355,636.752194
26597,360.000€,28512,4 hab,2 wc,189 m²,Villar del Olmo,40.3435,-3.2415,"(40.34346737484606, -3.2415042886969347)",18058.216754,899.348506,11851.252744


***
## Load past data to start training and estimate neighbourhood prices per m2 in 2030

In [3]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [4]:
df_past_mad= pd.read_csv('dataset_16_19_mad_3.csv')
df_past_mad

,year,neighbourhood,transactions,avg_eur_m2,avgsize
0,01-01-2016,Alcalá de Henares,1612.0,1427.0,94.43
1,01-01-2016,Alcobendas,804.0,2394.0,112.49
2,01-01-2016,Alcorcón,1096.0,1696.0,95.73
3,01-01-2016,Aranjuez,510.0,1373.0,95.26
4,01-01-2016,Arganda del Rey,537.0,1439.0,104.30
...,...,...,...,...,...
623,01-01-2019,San Sebastián de los Reyes,758.0,2314.0,102.00
624,01-01-2019,Torrejón de Ardoz,1281.0,1638.0,103.00
625,01-01-2019,Tres Cantos,605.0,2726.0,129.00
626,01-01-2019,Valdemoro,732.0,1592.0,101.00


In [5]:
df_past_mad['year']=pd.to_datetime(df_past_mad['year'])

In [6]:
index=range(0,157)
columns=['neighbourhood','transactions','avgsize','year_cat']

df_pred_past=pd.DataFrame(index=index, columns=columns)

df_pred_past['year_cat']=14
df_pred_past['neighbourhood']=df_past_mad['neighbourhood']
df_pred_past['transactions']=df_past_mad.groupby('neighbourhood')['transactions'].mean().values
df_pred_past['avgsize']=df_past_mad.groupby('neighbourhood')['avgsize'].mean().values

onehot_enc_pred = pd.get_dummies(df_pred_past['neighbourhood'])

df_pred_past = pd.concat([df_pred_past, onehot_enc_pred], axis=1, join="inner")

df_pred_past

,neighbourhood,transactions,avgsize,year_cat,Alcalá de Henares,Alcobendas,Alcorcón,Aranjuez,Arganda del Rey,Boadilla del Monte,...,Parla,Pinto,Pozuelo de Alarcón,Rivas-Vaciamadrid,San Fernando de Henares,San Sebastián de los Reyes,Torrejón de Ardoz,Tres Cantos,Valdemoro,Villaviciosa de Odón
0,Alcalá de Henares,1776.750000,93.465000,14,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Alcobendas,896.500000,108.745000,14,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Alcorcón,1271.000000,94.865000,14,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Aranjuez,651.750000,95.130000,14,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Arganda del Rey,636.000000,103.900000,14,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,Boadilla del Monte,471.750000,129.505000,14,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,Collado Villalba,744.250000,95.040000,14,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Colmenar Viejo,435.250000,99.290000,14,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Coslada,509.750000,96.660000,14,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Fuenlabrada,1237.250000,91.850000,14,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
labelencoder = LabelEncoder()

# Assigning numerical values and storing in another column
df_past_mad['year_cat'] = labelencoder.fit_transform(df_past_mad['year'])
df_past_mad

,year,neighbourhood,transactions,avg_eur_m2,avgsize,year_cat
0,2016-01-01,Alcalá de Henares,1612.0,1427.0,94.43,0
1,2016-01-01,Alcobendas,804.0,2394.0,112.49,0
2,2016-01-01,Alcorcón,1096.0,1696.0,95.73,0
3,2016-01-01,Aranjuez,510.0,1373.0,95.26,0
4,2016-01-01,Arganda del Rey,537.0,1439.0,104.30,0
...,...,...,...,...,...,...
623,2019-01-01,San Sebastián de los Reyes,758.0,2314.0,102.00,3
624,2019-01-01,Torrejón de Ardoz,1281.0,1638.0,103.00,3
625,2019-01-01,Tres Cantos,605.0,2726.0,129.00,3
626,2019-01-01,Valdemoro,732.0,1592.0,101.00,3


In [8]:
df_past_mad['transactions'] = df_past_mad['transactions'].replace('-','0')
df_past_mad['avgsize'] = df_past_mad['avgsize'].replace('-','0')
df_past_mad['avgsize'] = df_past_mad['avgsize'].replace(',','.')
df_past_mad['avg_eur_m2'] = df_past_mad['avg_eur_m2'].replace('-','0')
df_past_mad['avg_eur_m2'] = df_past_mad['avg_eur_m2'].replace(',','.')

df_past_mad=df_past_mad.fillna(0)

In [9]:
onehot_enc = pd.get_dummies(df_past_mad['neighbourhood'])

df_past_mad = pd.concat([df_past_mad, onehot_enc], axis=1, join="inner")

df_past_mad

,year,neighbourhood,transactions,avg_eur_m2,avgsize,year_cat,Alcalá de Henares,Alcobendas,Alcorcón,Aranjuez,...,Parla,Pinto,Pozuelo de Alarcón,Rivas-Vaciamadrid,San Fernando de Henares,San Sebastián de los Reyes,Torrejón de Ardoz,Tres Cantos,Valdemoro,Villaviciosa de Odón
0,2016-01-01,Alcalá de Henares,1612.0,1427.0,94.43,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2016-01-01,Alcobendas,804.0,2394.0,112.49,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2016-01-01,Alcorcón,1096.0,1696.0,95.73,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,2016-01-01,Aranjuez,510.0,1373.0,95.26,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,2016-01-01,Arganda del Rey,537.0,1439.0,104.30,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623,2019-01-01,San Sebastián de los Reyes,758.0,2314.0,102.00,3,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
624,2019-01-01,Torrejón de Ardoz,1281.0,1638.0,103.00,3,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
625,2019-01-01,Tres Cantos,605.0,2726.0,129.00,3,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
626,2019-01-01,Valdemoro,732.0,1592.0,101.00,3,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [10]:
scaler = MinMaxScaler()

df_past_mad[['transactions','avgsize']] = scaler.fit_transform(df_past_mad[['transactions','avgsize']])

df_past_mad

,year,neighbourhood,transactions,avg_eur_m2,avgsize,year_cat,Alcalá de Henares,Alcobendas,Alcorcón,Aranjuez,...,Parla,Pinto,Pozuelo de Alarcón,Rivas-Vaciamadrid,San Fernando de Henares,San Sebastián de los Reyes,Torrejón de Ardoz,Tres Cantos,Valdemoro,Villaviciosa de Odón
0,2016-01-01,Alcalá de Henares,0.840021,1427.0,0.470480,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2016-01-01,Alcobendas,0.418968,2394.0,0.560460,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2016-01-01,Alcorcón,0.571131,1696.0,0.476957,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,2016-01-01,Aranjuez,0.265763,1373.0,0.474615,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,2016-01-01,Arganda del Rey,0.279833,1439.0,0.519655,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623,2019-01-01,San Sebastián de los Reyes,0.394997,2314.0,0.508196,3,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
624,2019-01-01,Torrejón de Ardoz,0.667535,1638.0,0.513178,3,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
625,2019-01-01,Tres Cantos,0.315268,2726.0,0.642718,3,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
626,2019-01-01,Valdemoro,0.381449,1592.0,0.503214,3,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [11]:
df_pred_past[['transactions','avgsize']] = scaler.fit_transform(df_pred_past[['transactions','avgsize']])

df_pred_past= df_pred_past.drop(columns='neighbourhood')

df_pred_past

,transactions,avgsize,year_cat,Alcalá de Henares,Alcobendas,Alcorcón,Aranjuez,Arganda del Rey,Boadilla del Monte,Collado Villalba,...,Parla,Pinto,Pozuelo de Alarcón,Rivas-Vaciamadrid,San Fernando de Henares,San Sebastián de los Reyes,Torrejón de Ardoz,Tres Cantos,Valdemoro,Villaviciosa de Odón
0,1.000000,0.509207,14,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.503245,0.592454,14,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.714588,0.516835,14,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.365124,0.518278,14,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.356236,0.566058,14,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0.263544,0.705557,14,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,0.417325,0.517788,14,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
7,0.242946,0.540943,14,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0.284989,0.526614,14,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0.695542,0.500409,14,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## We split the train data into features and target and start the machine learning modelling to get the future predictions

In [12]:
y=np.array(df_past_mad['avg_eur_m2'])
X= df_past_mad.drop(columns=['avg_eur_m2','year','neighbourhood'], axis=1)

In [13]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.21,random_state=37,shuffle=True)

We start testing different models using this graph and moving towards the ones that gives better R2 rates
***
    
![Regression Models](https://miro.medium.com/max/1674/1*_Wx0vKokbXd20HlbLKpj2A.jpeg)

Start with a Linear Regression model

In [14]:
from sklearn.linear_model import LinearRegression

In [15]:
lr=LinearRegression()


In [16]:
lr.fit(x_train,y_train)

LinearRegression()

In [17]:
predictions_lr= lr.predict(x_test)

In [18]:
rmse= mean_squared_error(y_test,predictions_lr,squared=False)
r2_confidence = r2_score(y_test,predictions_lr)
mae=mean_absolute_error(y_test,predictions_lr)
mse=mean_squared_error(y_test,predictions_lr)
print('RMSE: ', rmse)
print('R2 confidence: ', r2_confidence)
print('MAE: ', mae)
print('MSE: ', mse)

RMSE:  407.0939299583674
R2 confidence:  0.9281160166758103
MAE:  287.8550465160857
MSE:  165725.46780894816


Now we fit the data with a Lasso Regression model

In [19]:
from sklearn.linear_model import Lasso

In [20]:
ls=Lasso(alpha=0.1)
ls.fit(x_train,y_train)


Lasso(alpha=0.1)

In [21]:
predictions_ls= ls.predict(x_test)

In [22]:
rmse= mean_squared_error(y_test,predictions_ls,squared=False)
r2_confidence = r2_score(y_test,predictions_ls)
mae=mean_absolute_error(y_test,predictions_ls)
mse=mean_squared_error(y_test,predictions_ls)
print('RMSE: ', rmse)
print('R2 confidence: ', r2_confidence)
print('MAE: ', mae)
print('MSE: ', mse)

RMSE:  407.96127156074493
R2 confidence:  0.9278093828352846
MAE:  287.70443744032906
MSE:  166432.39909345988


And now using a Ridge Regression model

In [23]:
from sklearn.linear_model import Ridge

In [24]:
rd=Ridge(alpha=0.1)
rd.fit(x_train,y_train)

Ridge(alpha=0.1)

In [25]:
predictions_rd=rd.predict(x_test)

In [26]:
rmse= mean_squared_error(y_test,predictions_rd,squared=False)
r2_confidence = r2_score(y_test,predictions_rd)
mae=mean_absolute_error(y_test,predictions_rd)
mse=mean_squared_error(y_test,predictions_rd)
print('RMSE: ', rmse)
print('R2 confidence: ', r2_confidence)
print('MAE: ', mae)
print('MSE: ', mse)

RMSE:  411.1241464067938
R2 confidence:  0.9266856732566794
MAE:  288.50032609993514
MSE:  169023.0637587148


Now with Polynomial Regression

In [27]:
from sklearn.linear_model import TheilSenRegressor

In [28]:
ts_reg=TheilSenRegressor()
ts_reg.fit(x_train,y_train)

TheilSenRegressor(max_subpopulation=10000)

In [29]:
predictions_ts_reg=ts_reg.predict(x_test)

In [30]:
rmse= mean_squared_error(y_test,predictions_ts_reg,squared=False)
r2_confidence = r2_score(y_test,predictions_ts_reg)
mae=mean_absolute_error(y_test,predictions_ts_reg)
mse=mean_squared_error(y_test,predictions_ts_reg)
print('RMSE: ', rmse)
print('R2 confidence: ', r2_confidence)
print('MAE: ', mae)
print('MSE: ', mse)

RMSE:  670.7319454488837
R2 confidence:  0.8048624429295024
MAE:  487.7765107240487
MSE:  449881.3426456444


Now with SVR model

In [31]:
from sklearn.svm import SVR

In [32]:
svr=SVR(kernel='linear', C=100, gamma=0.1)
svr.fit(x_train,y_train)

SVR(C=100, gamma=0.1, kernel='linear')

In [33]:
predictions_svr=svr.predict(x_test)

In [34]:
rmse= mean_squared_error(y_test,predictions_svr,squared=False)
r2_confidence = r2_score(y_test,predictions_svr)
mae=mean_absolute_error(y_test,predictions_svr)
mse=mean_squared_error(y_test,predictions_svr)
print('RMSE: ', rmse)
print('R2 confidence: ', r2_confidence)
print('MAE: ', mae)
print('MSE: ', mse)

RMSE:  1173.870966545356
R2 confidence:  0.40229952115089274
MAE:  823.634344415177
MSE:  1377973.0460981284


And lastly using Random Forest regression model

In [35]:
from sklearn.ensemble import RandomForestRegressor

In [36]:
random_for=RandomForestRegressor(n_estimators=300,max_features='log2',max_depth=70,random_state=42)
random_for.fit(x_train,y_train)

RandomForestRegressor(max_depth=70, max_features='log2', n_estimators=300,
                      random_state=42)

In [37]:
predictions_random_for=random_for.predict(x_test)

In [38]:
rmse= mean_squared_error(y_test,predictions_random_for,squared=False)
r2_confidence = r2_score(y_test,predictions_random_for)
mae=mean_absolute_error(y_test,predictions_random_for)
mse=mean_squared_error(y_test,predictions_random_for)
print('RMSE: ', rmse)
print('R2 confidence: ', r2_confidence)
print('MAE: ', mae)
print('MSE: ', mse)

RMSE:  747.4361540932547
R2 confidence:  0.7576789827080166
MAE:  591.590719614894
MSE:  558660.8044457155


***
## After testing three different models we see that the Linear Regression model is giving the best results in terms of scoring so we´ll use that one to estimate the future price of each neighbourhood/district in Madrid

In [39]:
pred_lr_future=lr.predict(df_pred_past)

df_pred_past['future_price_m2']=pred_lr_future

df_pred_past['neighbourhood']=df_past_mad['neighbourhood']

In [40]:
index=range(0,157)
columns=['neighbourhood','price_pred_2030']

price_pred_2030=pd.DataFrame(index=index, columns=columns)

price_pred_2030['neighbourhood']=df_past_mad['neighbourhood']
price_pred_2030['price_pred_2030']=pred_lr_future

price_pred_2030

,neighbourhood,price_pred_2030
0,Alcalá de Henares,5219.791626
1,Alcobendas,6167.352180
2,Alcorcón,5526.652352
3,Aranjuez,5128.460208
4,Arganda del Rey,5253.832835
5,Boadilla del Monte,6577.323787
6,Collado Villalba,5327.078220
7,Colmenar Viejo,5505.108851
8,Coslada,5537.057847
9,Fuenlabrada,5254.081977


In [386]:
# loading libraries
from sklearn import decomposition
from sklearn import linear_model
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler

# Creating an scaler object
std_slc = StandardScaler()

# Creating a pca object
pca = decomposition.PCA()

# Creating a linear regression object with an L2 penalty
lasso = linear_model.Lasso()

# Creating a pipeline of three steps. First, standardize the data.
# Second, tranform the data with PCA.
# Third, train a Decision Tree Classifier on the data.
pipe = Pipeline(steps=[('std_slc', std_slc),
                       ('pca', pca),
                       ('lasso', lasso)])

# Creating Parameter Space
# Create a list of a sequence of integers from 1 to 30 (the number of features in X + 1)
n_components = list(range(1,X.shape[1]+1,1))

# Creating lists of parameter for Lasso Regression
normalize = [True, False]
selection = ['cyclic', 'random']

# Creating a dictionary of all the parameter options 
# Note has you can access the parameters of steps of a pipeline by using '__’
parameters = dict(pca__n_components=n_components,
                  lasso__normalize=normalize,
                  lasso__selection=selection)

# Conducting Parameter Optmization With Pipeline
# Create a grid search object
clf = GridSearchCV(pipe, parameters)

# Fitting the grid search
clf.fit(X, y)

# Viewing The Best Parameters
print('Best Number Of Components:', clf.best_estimator_.get_params()['pca__n_components'])
print(clf.best_estimator_.get_params()['lasso'])

# Using Cross Validation To Evaluate Model
#CV_Result = cross_val_score(clf, X, y, cv=10, n_jobs=-1, scoring='r2')
#print(); print(CV_Result)
#print(); print(CV_Result.mean())
#print(); print(CV_Result.std())


ValueError: Parameter grid for parameter (pca__n_components) needs to be a list or numpy array, but got (<class 'int'>). Single values need to be wrapped in a list with one element.

In [382]:
clf.best_estimator_.get_params()

{'memory': None,
 'steps': [('std_slc', StandardScaler()),
  ('pca', PCA(n_components=157)),
  ('lasso', Lasso())],
 'verbose': False,
 'std_slc': StandardScaler(),
 'pca': PCA(n_components=157),
 'lasso': Lasso(),
 'std_slc__copy': True,
 'std_slc__with_mean': True,
 'std_slc__with_std': True,
 'pca__copy': True,
 'pca__iterated_power': 'auto',
 'pca__n_components': 157,
 'pca__random_state': None,
 'pca__svd_solver': 'auto',
 'pca__tol': 0.0,
 'pca__whiten': False,
 'lasso__alpha': 1.0,
 'lasso__copy_X': True,
 'lasso__fit_intercept': True,
 'lasso__max_iter': 1000,
 'lasso__normalize': False,
 'lasso__positive': False,
 'lasso__precompute': False,
 'lasso__random_state': None,
 'lasso__selection': 'cyclic',
 'lasso__tol': 0.0001,
 'lasso__warm_start': False}

In [485]:
Lasso(alpha=1.0,
      copy_X=True,
      fit_intercept=True,
      max_iter=1000,
      normalize=False,
      positive=False,
      precompute=False,
      random_state=None,
      selection='cyclic',
      tol=0.0001,
      warm_start=False)

Lasso()

***
## Now we load again the file we have with all the houses on the market in Madrid and add the estimated future price per m2 by district to get an estimation of the return of investment (ROI) per house

In [141]:
price_pred_2030['neighbourhood'] = price_pred_2030['neighbourhood'].str.replace('M-','')
price_pred_2030['neighbourhood'] = price_pred_2030['neighbourhood'].str.replace(' ','')

In [156]:
postal_code=pd.read_csv('listado_codigos_postales_mad.csv')

In [157]:
postal_code['poblacion']=postal_code['poblacion'].str.replace(' ','')

In [158]:
df_future_prices=pd.merge(postal_code,price_pred_2030,left_on='poblacion',right_on='neighbourhood',how='left',validate='many_to_one')
df_future_prices

,provincia,poblacion,codigopostalid,lat,lon,neighbourhood,price_pred_2030
0,Madrid,Recoletos,28001,-3.687601,40.408412,Recoletos,9744.692743
1,Madrid,NuevaEspaña,28002,-3.687601,40.408412,NuevaEspaña,8379.681217
2,Madrid,Trafalgar,28003,-3.687601,40.408412,Trafalgar,8406.019152
3,Madrid,Justicia,28004,-3.687601,40.408412,Justicia,9049.423811
4,Madrid,Imperial,28005,-3.687601,40.408412,Imperial,7304.519868
5,Madrid,Castellana,28006,-3.687601,40.408412,Castellana,9726.193015
6,Madrid,Adelfas,28007,-3.687601,40.408412,Adelfas,7429.328444
7,Madrid,Argüelles,28008,-3.687601,40.408412,Argüelles,6518.456586
8,Madrid,Ibiza,28009,-3.687601,40.408412,Ibiza,8436.403193
9,Madrid,Almagro,28010,-3.687601,40.408412,Almagro,8694.638812


In [159]:
df_future_prices=df_future_prices.drop(columns=['provincia','poblacion','lat','lon','neighbourhood'])
df_future_prices

,codigopostalid,price_pred_2030
0,28001,9744.692743
1,28002,8379.681217
2,28003,8406.019152
3,28004,9049.423811
4,28005,7304.519868
5,28006,9726.193015
6,28007,7429.328444
7,28008,6518.456586
8,28009,8436.403193
9,28010,8694.638812


In [160]:
df_final_houses=pd.merge(df_mad_houses,df_future_prices,left_on='post_code',right_on='codigopostalid',how='left',validate='many_to_many')
df_final_houses

,Unnamed: 0,prices,post_code,rooms,bathrooms,size,poblacion,lat,lon,latlon_merged,transport,school,health_centre,codigopostalid,price_pred_2030
0,0,112.062€,28031,1 hab,1 wc,49 m²,VillaVallecas,40.377436,-3.624158,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674,892.420899,28031,6523.040826
1,1,137.000€,28031,1 hab,1 wc,32 m²,VillaVallecas,40.377436,-3.624158,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674,892.420899,28031,6523.040826
2,2,100.000€,28031,3 hab,1 wc,48 m²,VillaVallecas,40.377436,-3.624158,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674,892.420899,28031,6523.040826
3,3,100.000€,28031,3 hab,1 wc,48 m²,VillaVallecas,40.377436,-3.624158,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674,892.420899,28031,6523.040826
4,4,135.000€,28031,1 hab,1 wc,49 m²,VillaVallecas,40.377436,-3.624158,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674,892.420899,28031,6523.040826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28067,26595,175.000€,28721,4 hab,2 wc,156 m²,CabanillasdelaSierra,40.817980,-3.605480,"(40.81798043643363, -3.6054796201378334)",23994.341683,1790.021114,5226.267593,28721,NaN
28068,26596,329.000€,28791,3 hab,2 wc,150 m²,SotodelReal,40.752560,-3.779915,"(40.752559745929474, -3.7799147561052875)",11952.570673,307.756355,636.752194,28791,NaN
28069,26597,360.000€,28512,4 hab,2 wc,189 m²,VillardelOlmo,40.343467,-3.241504,"(40.34346737484606, -3.2415042886969347)",18058.216754,899.348506,11851.252744,28512,NaN
28070,26598,450.000€,28749,3 hab,3 wc,195 m²,Rascafría,40.922484,-3.820778,"(40.92248427948072, -3.8207777728723684)",16447.682265,4113.975968,5249.495580,28749,NaN


In [165]:
df_final_houses['rooms'] = df_final_houses['rooms'].str.replace(' hab','')
df_final_houses['bathrooms'] = df_final_houses['bathrooms'].str.replace(' wc','')
df_final_houses['size'] = df_final_houses['size'].str.replace(' m²','')
df_final_houses['prices'] = df_final_houses['prices'].str.replace('.',',')
df_final_houses['prices'] = df_final_houses['prices'].str.replace(',','')
df_final_houses['prices'] = df_final_houses['prices'].str.replace('€','')
df_mad_houses['poblacion'] = df_mad_houses['poblacion'].str.replace(' ','')

df_final_houses

,Unnamed: 0,prices,post_code,rooms,bathrooms,size,poblacion,lat,lon,latlon_merged,transport,school,health_centre,codigopostalid,price_pred_2030
0,0,"112,062",28031,1,1,49,VillaVallecas,40.377436,-3.624158,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674,892.420899,28031,6523.040826
1,1,"137,000",28031,1,1,32,VillaVallecas,40.377436,-3.624158,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674,892.420899,28031,6523.040826
2,2,"100,000",28031,3,1,48,VillaVallecas,40.377436,-3.624158,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674,892.420899,28031,6523.040826
3,3,"100,000",28031,3,1,48,VillaVallecas,40.377436,-3.624158,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674,892.420899,28031,6523.040826
4,4,"135,000",28031,1,1,49,VillaVallecas,40.377436,-3.624158,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674,892.420899,28031,6523.040826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28067,26595,"175,000",28721,4,2,156,CabanillasdelaSierra,40.817980,-3.605480,"(40.81798043643363, -3.6054796201378334)",23994.341683,1790.021114,5226.267593,28721,NaN
28068,26596,"329,000",28791,3,2,150,SotodelReal,40.752560,-3.779915,"(40.752559745929474, -3.7799147561052875)",11952.570673,307.756355,636.752194,28791,NaN
28069,26597,"360,000",28512,4,2,189,VillardelOlmo,40.343467,-3.241504,"(40.34346737484606, -3.2415042886969347)",18058.216754,899.348506,11851.252744,28512,NaN
28070,26598,"450,000",28749,3,3,195,Rascafría,40.922484,-3.820778,"(40.92248427948072, -3.8207777728723684)",16447.682265,4113.975968,5249.495580,28749,NaN


In [182]:
df_final_houses.drop(columns='Unnamed: 0',inplace=True)

df_final_houses['prices']=pd.to_numeric(df_final_houses['prices'])
df_final_houses['rooms']=pd.to_numeric(df_final_houses['rooms'])
df_final_houses['bathrooms']=pd.to_numeric(df_final_houses['bathrooms'])
df_final_houses['size']=pd.to_numeric(df_final_houses['size'])

In [187]:
#Create a calculation for small neighbourhoods where we don´t have an estimation of future price. We fill those with an increased 8% of the value in 2021
df_final_houses['price_pred_2030'].fillna(df_final_houses['prices']/df_final_houses['size']*1.08,inplace=True)

In [189]:
df_final_houses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28072 entries, 0 to 28071
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       28072 non-null  int64  
 1   prices           28072 non-null  int64  
 2   post_code        28072 non-null  int64  
 3   rooms            28072 non-null  int64  
 4   bathrooms        28072 non-null  int64  
 5   size             28072 non-null  int64  
 6   poblacion        28072 non-null  object 
 7   lat              28072 non-null  float64
 8   lon              28072 non-null  float64
 9   latlon_merged    28072 non-null  object 
 10  transport        28072 non-null  float64
 11  school           28072 non-null  float64
 12  health_centre    28072 non-null  float64
 13  codigopostalid   28072 non-null  int64  
 14  price_pred_2030  28072 non-null  float64
dtypes: float64(6), int64(7), object(2)
memory usage: 3.4+ MB
